In [5]:
import json
import logging
import re
import warnings
from pathlib import Path
from pprint import pprint, pformat
from typing import Any, Literal

import numpy as np
import numpy.typing as npt
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
fp: str = "../../../../Documents/data_dump/bike_data/database.parquet"
data: pl.DataFrame = pl.read_parquet(fp)
console.print(f"Shape: {data.shape}", style="info")

data.head()

Shape: (13903, 16)

datetime,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64
"""2011-01-01 00:00:00""",1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
"""2011-01-01 01:00:00""",1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0.0,8,32,40
"""2011-01-01 02:00:00""",1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0.0,5,27,32
"""2011-01-01 03:00:00""",1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
"""2011-01-01 04:00:00""",1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [9]:
fp: str = "../bike_data.parquet"
data_table: pl.DataFrame = pl.read_parquet(fp)
console.print(f"Shape: {data_table.shape}", style="info")

data_table.head(200)

Shape: (120, 16)

datetime,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
datetime[μs],i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64
2011-01-01 00:00:00,4,0,1,0,1,5,0,2,0.2422,0.2647,0.8092,0.1283,1,0,1
2011-01-01 01:00:00,4,0,1,1,1,5,0,2,0.2977,0.2721,0.7112,0.378,1,1,2
2011-01-01 02:00:00,4,0,1,2,1,5,0,1,0.3232,0.3343,0.3255,0.2483,1,1,2
2011-01-01 03:00:00,4,0,1,3,1,5,0,1,0.4511,0.4465,0.6114,0.0584,1,2,3
2011-01-01 04:00:00,4,0,1,4,1,5,0,1,0.4426,0.5069,0.687,0.0977,1,1,2
2011-01-01 05:00:00,4,0,1,5,1,5,0,1,0.4516,0.4264,0.3908,0.2009,2,2,4
2011-01-01 06:00:00,4,0,1,6,1,5,0,2,0.4333,0.4368,0.6865,0.3047,2,2,4
2011-01-01 07:00:00,4,0,1,7,1,5,0,1,0.5494,0.5683,0.5821,0.2342,4,6,10
2011-01-01 08:00:00,4,0,1,8,1,5,0,1,0.6458,0.6198,0.4884,0.1696,11,11,22
